In [10]:
import pickle
import json
from pprint import pprint
from mmdet.datasets import VOCTXTDataset
import sys

In [11]:
data_root = 'data/VOCdevkit/'
ann_file=data_root + 'VOC2012/ImageSets/Main/train_unlabeled.txt',
img_prefix='data/VOCdevkit/VOC2012/'

In [12]:
# !./tools/dist_test.sh configs/pascal_voc/ssd300_voc.py work_dirs/ssd300_voc/epoch_24.pth 8 --out results.pkl --eval bbox

In [13]:
def extract_boxes(fname):
        with open(fname) as f:
            content = f.readlines()
            f.close()
            content = [x.strip().split(' ') for x in content]
            return content

In [14]:
# for each image 20xnumber of boxes

with open('results.pkl', 'rb') as f:
    data = pickle.load(f)

In [7]:
# len(data)

9290

In [16]:
VOCTXTDataset.CLASSES
cat2label = {cat: i for i, cat in enumerate(VOCTXTDataset.CLASSES)}
label2cat = {i: cat for  i, cat in enumerate(VOCTXTDataset.CLASSES)}
print('Loading JSON ... ... ... \n')
with open('results.pkl.bbox.json') as f:
    data = json.load(f)
merg = {}
for index, pred in enumerate(data):
    img_id = pred['image_id']
    if not (img_id in merg):
        merg[img_id] = list()
    merg[img_id].append(pred)

files = merg.keys()
for index, file in enumerate(files):
    line = ''
    for box in merg[file]:
        if box['category_id'] in label2cat.keys():
            category_label = label2cat[box['category_id']] if box['category_id'] in label2cat.keys() else 'background'
            bbox = box['bbox']
            info = extract_boxes(f'{img_prefix}/Annotations/{file}.txt')
            width = int(info[0][1])
            height = int(info[0][2])
            line = f"{line}{category_label} {width} {height} 0 {int(bbox[0])} {int(bbox[1])} {int(bbox[0]+bbox[2])} {int(bbox[3]+bbox[1])}\n"
        else:
            print(f"\n {box['category_id']} \n")
            
    sys.stdout.write(f"\r [ {file} ] {index} / {len(files)}")
    sys.stdout.flush() 
#     img_id = f"{pred['image_id']}_psudo_labels"

    f = open(f'{img_prefix}/Annotations/{file}_psudo_labels.txt', 'w')
    f.write(line)
    f.close()

Loading JSON ... ... ... 

 [ 2008_003613 ] 9289 / 9290